<a href="https://colab.research.google.com/github/cml096/GoogleColabRepo/blob/master/Project_Cap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [126]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

In [127]:
df = pd.read_excel('Rendimiento CMP V2.xlsx', sheet_name='MinaMLC')

#Preparando Data

##Filtro

Filtro 1

In [128]:
df_ = df[df['División de personal'] == 'CMP VALLE DEL HUASCO']

Filtro 2

In [129]:
df_test_minera_colorado = df_[df_['Subdivisión de personal'] == 'MINA LOS COLORA']

##Variables no utilizables

In [130]:
df_ = df_test_minera_colorado

In [131]:
filtro = ['División de personal','Cod. División','Subdivisión de personal','Cod. Subdivisión','Cod. Unidad','Cod. Posición','Fecha de nacimiento','Fecha de alta','Status empleado para Gestión tiempos per']

In [132]:
df_ = df_.drop(filtro,axis=1)

##Divición Dia

Cambiamos nombre columna "Dia" por "Fecha"

In [133]:
df_ = df_.rename(columns={'Dia':'Fecha'})

Divición de la variable "Dia"

In [134]:
def dia(fecha):
  aux = str(fecha.date()).split('-')
  dia = str(aux[2])
  return dia

def mes(fecha):
  aux = str(fecha.date()).split('-')
  mes = str(aux[1])
  return mes

def ano(fecha):
  aux = str(fecha.date()).split('-')
  ano = str(aux[0])
  return ano

Anexamos columna "Dia", "Mes" y "Año"

In [135]:
df_['Dia'] = df_.apply(lambda row : dia(row['Fecha']),axis=1)
df_['Mes'] = df_.apply(lambda row : mes(row['Fecha']),axis=1)
df_['Año'] = df_.apply(lambda row : ano(row['Fecha']),axis=1)

##Columna "Eficiencia"

In [137]:
filtro = ['Fecha','Turno','Cargas (u)','Movimiento (t)','tdem (seg)',	'R-ef (t/h)','Número de personal','Edad del empleado','Equipo']
df_aux = df_.drop(filtro, axis=1)

Agrupamos por "unidad organizativa" y lo almacenamos en un data frame

In [138]:
group = df_aux.groupby(['Unidad organizativa']).max()
df_group = pd.DataFrame(group)

Creamos un array que contiene las tefe max de cada Unidad organizativa

In [139]:
def genera(data):
  lista = []
  for index, row in data.iterrows():
    lista.append([index,row['tefe (seg)']])
  return lista

In [140]:
max_por_unidad = genera(df_group)
len(max_por_unidad)

15

Agregamos la columna "Eficiencia" que generaremos con la furmula
* tefe(x tupla)*100/tefe_max

In [165]:
def rellena(data,lista):
  aux = []
  for index, row in data.iterrows():
    unidad = row['Unidad organizativa']
    tefe = row['tefe (seg)']
    for unidad_max,tefe_max in lista:
      if unidad == unidad_max:
        resultado = (tefe)/tefe_max
        aux.append(resultado)
  return aux

In [170]:
eficiencia = rellena(df_,max_por_unidad)
len(eficiencia)

1825

Le agregamos una columna al data frame

In [143]:
df_['Eficiencia'] = eficiencia

##Columna "Test" binaria

In [144]:
filtro = ['Eficiencia']
df_aux = df_.drop(filtro, axis=1)

In [145]:
group = df_aux.groupby(['Unidad organizativa']).median()
df_group = pd.DataFrame(group)

In [160]:
media_por_unidad = genera(df_group)
len(media_por_unidad)

15

In [166]:
def rellena_binario(data,lista):
  aux = []
  for index, row in data.iterrows():
    unidad = row['Unidad organizativa']
    tefe = row['tefe (seg)']
    for unidad_median,tefe_median in lista:
      if unidad == unidad_median:
        if tefe >= tefe_median:
          aux.append(1)
        elif tefe < tefe_median:
          aux.append(0)
  return aux

In [167]:
eficiencia_binaria = rellena_binario(df_,media_por_unidad)
len(eficiencia_binaria)

1825

In [168]:
df_['Eficiencia (bin)'] = eficiencia_binaria

#Regresión Logística

Generamos nuestro data de entrenamiento y le eliminamos las columnas que no ocuparemos en el modolamiento

In [211]:
filtro = ['tefe (seg)','tdem (seg)','R-ef (t/h)','Número de personal','Grupo de personal','Mes','Dia','Año','Eficiencia']
train = df_.drop(filtro,axis=1)
train.head()

,Fecha,Turno,Equipo,Cargas (u),Movimiento (t),Unidad organizativa,Posición,Edad del empleado,Eficiencia (bin)
0,2019-07-01,2,211,93,21134.20117,MANTENIMIENTO MINA,ELECTRICO III,29,0
1,2019-07-01,2,213,12,2833.59961,MANTENIMIENTO MINA,MECANICO I,54,0
4,2019-07-01,2,307,100,23335.70117,ADMINISTRACION SUPTCIA PLANTAS,OPERADOR ELECTRICO II PLANTA,62,1
5,2019-07-02,1,211,148,34061.60938,CAMIONES CAT 793F MLC,OPERADOR III,48,1
6,2019-07-02,1,213,7,1632.20007,MANTENIMIENTO MINA,MECANICO III,33,0


Creamos nuestras variables dummies

In [204]:
unidad = pd.get_dummies(df_['Unidad organizativa'],drop_first=True)
posicion = pd.get_dummies(df_['Posición'],drop_first=True)
#fecha = pd.get_dummies(df_['Fecha'],drop_first=True)

**DUDA** Equipo, turno, edad, es correcto tenerlas como una variable de tipo entera (int) dado que en el modelo las quiero ocupar como variables categoricas.

In [205]:
train.drop(['Fecha','Unidad organizativa','Posición'],axis=1,inplace=True)
#train = pd.concat([train,unidad,posicion])

In [206]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1825 entries, 0 to 3137
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Turno              1825 non-null   int64  
 1   Equipo             1825 non-null   int64  
 2   Cargas (u)         1825 non-null   int64  
 3   Movimiento (t)     1825 non-null   float64
 4   Edad del empleado  1825 non-null   int64  
 5   Eficiencia (bin)   1825 non-null   int64  
dtypes: float64(1), int64(5)
memory usage: 179.8 KB


Creamos nuestra data de entrenamiento y de test

In [207]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train.drop('Eficiencia (bin)',axis=1),train['Eficiencia (bin)'],test_size=0.30,random_state=101)

Importamos el modelo, creamos la regresión y entrenamos con nuestra data de entrenamiento

In [208]:
from sklearn.linear_model import LogisticRegression
modelo = LogisticRegression()
modelo.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

Predicción con la data de test

In [209]:
y_pred = modelo.predict(X_test)

Evaluacion del modelo

In [210]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.82      0.77      0.79       267
           1       0.79      0.84      0.81       281

    accuracy                           0.80       548
   macro avg       0.81      0.80      0.80       548
weighted avg       0.81      0.80      0.80       548



#Random Forest